In [1]:
from xgboost import XGBClassifier
import anndata as an
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 

import sys
sys.path.append('../src')
from models.lib.lightning_train import DataModule
import xgboost as xgb

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
module = DataModule(
    datafiles=['../data/retina/retina_T.h5ad'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    index_col='cell',
    batch_size=16,
    num_workers=0,
    shuffle=True,
    drop_last=False,
    normalize=False,
)

module.setup()

Creating train/val/test DataLoaders...


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Done, continuing to training.


In [3]:
loader = module.trainloader

In [4]:
sample = next(iter(loader))
X = sample[0].numpy()
y = sample[1].numpy()

In [7]:
from sklearn.metrics import accuracy_score

model = XGBClassifier()
train = module.trainloader
test = module.valloader

# Train on one minibatch to get started 
sample = next(iter(loader))
X = sample[0].numpy()
y = sample[1].numpy()

model = model.fit(X, y)

[22:29:59] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
for i, (trainsample, valsample) in enumerate(zip(train, test)):
    if i == 5:
        break
        
    X_train, y_train = trainsample 
    X_test, y_test = valsample
    
    model = model.fit(X_train, y_train, xgb_model=model.get_booster())
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)